# Parcial 1: Indicaciones.


Para la base de datos https://data.world/exercises/linear-regression-exercise-1/workspace/file?filename=cancer_reg.csv determinar las variables predictoras (no necesariamente todas las existentes en la base de datos) y una variable a predecir. Realizar una regresión múltiple para el caso, y probar el modelo con y sin preprocesamiento de acuerdo a las características de los datos seleccionados. Reportar el error de entrenamiento y validación o de validación cruzada según la estrategia seleccionada. Justificar todos los supuestos y las decisiones tomadas. Concluir sobre los resultados.


In [2]:
#Importamos las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [1]:
#Conectar con drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Se carga el archivo CSV
data = pd.read_csv('/content/drive/MyDrive/Aprendizaje Supervisado/Bases de Datos/cancer_reg.csv')
#Se visualizan los datos
print(data.head())
print(data.describe())

   avganncount  avgdeathsperyear  target_deathrate  incidencerate  medincome  \
0       1397.0               469             164.9          489.8      61898   
1        173.0                70             161.3          411.6      48127   
2        102.0                50             174.7          349.7      49348   
3        427.0               202             194.8          430.4      44243   
4         57.0                26             144.4          350.1      49955   

   popest2015  povertypercent  studypercap           binnedinc  medianage  \
0      260131            11.2   499.748204   (61494.5, 125635]       39.3   
1       43269            18.6    23.111234  (48021.6, 51046.4]       33.0   
2       21026            14.6    47.560164  (48021.6, 51046.4]       45.0   
3       75882            17.1   342.637253    (42724.4, 45201]       42.8   
4       10321            12.5     0.000000  (48021.6, 51046.4]       48.3   

   ...  pctprivatecoveragealone  pctempprivcoverage pctp

In [4]:
#Visualizmaos el nombre de las columnas
data.columns

Index(['avganncount', 'avgdeathsperyear', 'target_deathrate', 'incidencerate',
       'medincome', 'popest2015', 'povertypercent', 'studypercap', 'binnedinc',
       'medianage', 'medianagemale', 'medianagefemale', 'geography',
       'percentmarried', 'pctnohs18_24', 'pcths18_24', 'pctsomecol18_24',
       'pctbachdeg18_24', 'pcths25_over', 'pctbachdeg25_over',
       'pctemployed16_over', 'pctunemployed16_over', 'pctprivatecoverage',
       'pctprivatecoveragealone', 'pctempprivcoverage', 'pctpubliccoverage',
       'pctpubliccoveragealone', 'pctwhite', 'pctblack', 'pctasian',
       'pctotherrace', 'pctmarriedhouseholds', 'birthrate'],
      dtype='object')

In [22]:
#Seleccionamos las variables predictorias y la variable objetivo
Vp= ['incidencerate', 'pctprivatecoverage', 'pctpubliccoverage', 'medincome', 'povertypercent']
Vo= ['target_deathrate']

In [23]:
#Separamos las variables predictorias y objetivo
X = data[Vp]
y = data[Vo]

In [24]:
#Dividimos los datos en conjuntos de entrenamiento y prueba (80% - 20%)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
#Instanciamos y creamos el modelo de regresión lineal
model = LinearRegression() #Intanciamos
model.fit(X_train, y_train) #Entrenamos el modelo

LinearRegression()

In [26]:
#Miramos el score para saber el rendimiento del modelo
model.score(X, y)

0.4200453177440565

In [27]:
#Evaluamos el modelo son preprocesamiento
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

#Calculamos el error
from sklearn.metrics import mean_squared_error
train_error = mean_squared_error(y_train, y_pred_train)
test_error= mean_squared_error(y_test, y_pred_test)

print("Error de entrenamiento:", train_error)
print("Error de prueba:", test_error)

Error de entrenamiento: 441.8890590471096
Error de prueba: 464.93813310627337


In [28]:
#Preprocesamiento
pipe = Pipeline([('poly', PolynomialFeatures()),
                 ('model', LinearRegression(fit_intercept=True))])

In [31]:
#Entrenamos el modelo con preprocesamiento
pipe.fit(X_train, y_train)

Pipeline(steps=[('poly', PolynomialFeatures()), ('model', LinearRegression())])

In [32]:
#Evaluar el modelo con preprocesamiento
y_pred_train_poly = pipe.predict(X_train)
y_pred_test_train = pipe.predict(X_test)

In [33]:
#Calcular el MSE con preprocesamiento
train_error_poly = mean_squared_error(y_train, y_pred_train_poly)
test_error_poly = mean_squared_error(y_test, y_pred_test_train)

print("Error de entrenamiento con preprocesamiento:", train_error_poly)
print("Error de prueba con preprocesamiento:", test_error_poly)

Error de entrenamiento con preprocesamiento: 407.1848922589658
Error de prueba con preprocesamiento: 430.5780118422436


In [34]:
#Validación cruzada y ajuste del grado del polinomio
#Definimos los diferentes grados de polonimo
poly_degrees = [1, 2, 3, 4, 5]

In [37]:
#vamos a crear una lista vacía para almacenas los scores que resulten de evaluar las regresiones polinomicas con cada uno de los grados
scores_train = []
scores_test = []

In [41]:
# Probar diferentes grados de polinomio
from sklearn.model_selection import cross_val_score
for degree in poly_degrees:
    # Crear un pipeline que estandarice los datos y aplique regresión polinómica
    pipeline = Pipeline([
        ('poly_features', PolynomialFeatures(degree=degree)),
        ('regressor', LinearRegression())
    ])
    #calcular el score de validación cruzada
    scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error') #Removed extra indent
    scores_train.append(-np.mean(scores))

In [44]:
scores.mean()

-4091.59769189983

In [45]:
score_RMS = cross_val_score(LinearRegression(fit_intercept = True),
                        X, y, cv = 5, scoring = "neg_mean_squared_error")

In [46]:
-score_RMS.mean()

457.41688706813085

##Análisis del Proceso:

1. **Modelo sin preprocesamiento:**

Como primer momento el modelo tiene un score de 42% con un error de entrenamiento de 441.89 y el error de prueba de 464.93. Esto indica que el modelo se ajusta con una pequeña diferencia.

2. **Preprocesamiento:**

Después se realizó el preprocesamiento de los datos usando una regresión polinomica y se obtuvieron los datos en error de entramiento de 407.18 y error de prueba 430.57, lo que indica que la transformación polínómica toma mejor la relación entre las variables.

3. **Validación cruzada**:

AL probar los diferentes grados del polinomo, los errores medios -4091.59 y 457.41 para el ajuste) indican que el grado de polinomio y la validación cruzada han ayudado a reducir el error cuadrático medio, obteniendo una estimación más precisa. Esto es particularmente útil para evitar sobreajuste.
